In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os

In [0]:
os.chdir('gdrive/My Drive/data')

In [4]:
!pip install ktrain

In [0]:
import ktrain
from ktrain import text
import pandas as pd
import tensorflow as tf
import timeit
import argparse

from utils import load_data, train_dev_test, make_map_from_nested, word_vectors, trigram_vectors, all_vectors

In [0]:
def available_device(device_name):
    if device_name != '/device:GPU:0':
        print('Calculations will be made on CPU.')
    else:
        print('Calculations will be made on GPU.')
    return device_name

In [0]:
seed = 42
database, _ = load_data('./')

In [0]:
def available_device(device_name):
    if device_name != '/device:GPU:0':
        print('Calculations will be made on CPU.')
    else:
        print('Calculations will be made on GPU.')
    return device_name

In [0]:
database = database.drop('username', axis=1).rename(columns={"text": "TEXT", "class": "LABEL"})

In [0]:
max_words = max([len(t.split()) for t in database.TEXT])

In [11]:
max_words

30

In [0]:
def train(device_name, lr, epochs, batch, weight_save_path, df, text_column_df_name, possible_targets_list,
                predictor_save_path, maxlen=450):
    with tf.device(device_name):
        (x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(df, text_column_df_name,
                                                                           possible_targets_list,
                                                                           preprocess_mode='bert', maxlen=maxlen)
        model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)
        learner = ktrain.get_learner(model, train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=batch)
        # learner.fit_onecycle(lr, epochs, checkpoint_folder=weight_save_path)
        learner.autofit(lr, epochs, checkpoint_folder=weight_save_path)
    predictor = ktrain.get_predictor(learner.model, preproc)
    predictor.save(predictor_save_path)

In [13]:
lr = 2e-5
batch = 12
epochs = 1
text_column_name = 'TEXT'
possible_targets_list = list(set((database.LABEL)))
weight_save_path = './bert_weights'
predictor_save_path = './my_predictor'
device_name = tf.test.gpu_device_name()
possible_device = available_device(device_name)

Calculations will be made on GPU.


In [14]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(database, text_column_name,
                                                                   label_columns=['LABEL'], lang='ru',
                                                                   preprocess_mode='bert', maxlen=max_words)

preprocessing train...
language: ru


preprocessing test...
language: ru


In [15]:
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 30
done.


In [0]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=batch)

In [17]:
learner.autofit(lr, epochs, checkpoint_folder=weight_save_path)



begin training using triangular learning rate policy with max lr of 2e-05...
Train on 170840 samples, validate on 18983 samples
170840/170840 [==============================] - 13662s 80ms/sample - loss: 0.7776 - accuracy: 0.7574 - val_loss: 0.5557 - val_accuracy: 0.8237


In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save(predictor_save_path)